In [1]:
import sympy
import numpy as np

In [2]:
p_t0, f0_t0, f1_t0, f2_t0, p_t1, f0_t1, f1_t1, f2_t1 = [np.array(sympy.symbols(f"x{i}{t} y{i}{t} z{i}{t}")) for t in range(2) for i in range(4)]
# display(p_t0, f0_t0, f1_t0, f2_t0, p_t1, f0_t1, f1_t1, f2_t1)

In [3]:
t = sympy.Symbol("t")
p = (p_t1 - p_t0) * t + p_t0
f0 = (f0_t1 - f0_t0) * t + f0_t0
f1 = (f1_t1 - f1_t0) * t + f1_t0
f2 = (f2_t1 - f2_t0) * t + f2_t0
# display(p, f0, f1, f2)

In [4]:
n = np.cross(f1 - f0, f2 - f0)
q = p - f0
d = np.dot(n, q).simplify()
A, B, C, D = sympy.poly(d, t).simplify().coeffs()

In [5]:
(d - (A * t**3 + B * t**2 + C * t + D)).simplify() == 0

True

In [6]:
data = {}
np.random.seed(21)
data["x00"], data["y00"], data["z00"] = np.random.rand(3)
data["x01"], data["y01"], data["z01"] = np.random.rand(3)

data["x10"], data["y10"], data["z10"] = np.random.rand(3)
data["x11"], data["y11"], data["z11"] = np.random.rand(3)

data["x20"], data["y20"], data["z20"] = np.random.rand(3)
data["x21"], data["y21"], data["z21"] = np.random.rand(3)

data["x30"], data["y30"], data["z30"] = np.random.rand(3)
data["x31"], data["y31"], data["z31"] = np.random.rand(3)

In [7]:
A.subs(data) * t**3 + B.subs(data) * t**2 + C.subs(data) * t + D.subs(data)

-0.669273980156659*t**3 + 1.24958157106678*t**2 - 0.605823657030843*t + 0.0717055064872666

In [8]:
import plotly.graph_objects as go

d = A.subs(data) * t**3 + B.subs(data) * t**2 + C.subs(data) * t + D.subs(data)

ts = np.linspace(-0.1, 1.1, 100)
ds = np.array([d.subs({"t": t}) for t in ts], dtype=np.float64)
max_d = np.max(np.abs(ds))

from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=ts, y=ds, name="d(t)"))
fig.add_trace(
    go.Scatter(x=ts, y=np.array([d.diff("t").subs({"t": t}) for t in ts], dtype=np.float64), name="d'(t)"), 
    secondary_y=True)
fig.update_layout(
    template="plotly_white",
    width=1200,
    height=800,
    # margin=dict(l=0, r=0, b=0, t=0),
    xaxis=dict(
        range=[-0.1, 1.1],
        # scaleanchor="y",
        # scaleratio=1,
    ),
    yaxis=dict(
        # range=[ds.min(), ds.max()],
        # scaleanchor="x",
        # scaleratio=1,
    ),
)
fig.add_hline(y=0, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0, line_width=1, line_dash="dash", line_color="black")
fig.add_vline(x=1, line_width=1, line_dash="dash", line_color="black")

aspect_ratio = 800/1200 * (ts.max() - ts.min()) / (ds.max() - ds.min())
delta = 0.02 * (ds.max() - ds.min())
dx = aspect_ratio * delta
dy = delta
for toi in sympy.roots(d):
    if not toi.is_real or not (0 <= toi <= 1):
        continue
    toi = float(toi)
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=toi-dx, y0=-dy, x1=toi+dx, y1=dy,
        line_color="LightSeaGreen",
    )

for tt in sympy.roots(d.diff(t)):
    dd = float(d.subs({"t": tt}))
    tt = float(tt)
    fig.add_shape(type="circle", xref="x", yref="y",
        x0=tt-dx, y0=dd-dy, x1=tt+dx, y1=dd+dy,
        line_color="sandybrown",
    )

ti = 0
newton_x = []
newton_y = []
while list(sympy.roots(d))[0] - ti > 1e-12:
    newton_x.append(ti)
    newton_y.append(float(d.subs(dict(t=ti))))
    m = float(d.diff(t).subs(dict(t=ti)))
    newton_x.append(ti - newton_y[-1] / m)
    newton_y.append(0)
    ti = newton_x[-1]
print(len(newton_x) // 2)
fig.add_trace(go.Scatter(
    x=newton_x,
    y=newton_y,
    mode="lines",
    name="Newton's method")
)

ti = 0
newton_x = []
newton_y = []
m = float(d.diff(t).subs(dict(t=ti)))
while list(sympy.roots(d))[0] - ti > 1e-12:
    newton_x.append(ti)
    newton_y.append(float(d.subs(dict(t=ti))))
    newton_x.append(ti - newton_y[-1] / m)
    newton_y.append(0)
    ti = newton_x[-1]
print(len(newton_x) // 2)
fig.add_trace(go.Scatter(
    x=newton_x,
    y=newton_y,
    mode="lines",
    name="modified method")
)

fig.show()

5
54


In [9]:
t0, t1 = 0, 1
tm0, tm1 = sympy.roots(d.diff(t))
assert(tm0 < tm1)
if A.subs(data) > 0:
    print("Case 1")
    tmin = tm1
    tmax = tm0

    d_tmin = d.subs({"t": tmin})
    if d_tmin > 0:
        print("no collision")
    elif t0 < tmin:
        print(f"possible collision in [{tmax}, {tmin}]")
    else:
        print("no collision")
else:
    assert(A.subs(data) < 0)
    tmin = tm0
    tmax = tm1
    if d.subs({"t": tmin}) >= 0:
        print("Case 2")
        if tmax < t1:
            print(f"possible collision in ({tmax}, {t1}]")
        else:
            print("no collision")
    else:
        print("Case 3")
        if t0 < tmin:
            print(f"possible collision in ({t0}, {tmin})")
        elif tmax < t1:
            print(f"possible collision in ({tmax}, {t1}]")
        else:
            print("no collision")

Case 3
possible collision in (0, 0.329788517362415)
